In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from tqdm import tqdm
import pandas as pd

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# Load tokenizer and model from the dataset path
model_path = "/kaggle/input/deberta-v3-large-gptlastoptimisedepoch3"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Ensure model is in evaluation mode
model.eval()


DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 1024, padding_idx=0)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-23): 24 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNo

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
model.to(device)

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 1024, padding_idx=0)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-23): 24 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNo

In [5]:
id_to_label = {
    0: '1.0',
    1: '1.5',
    2: '2.0',
    3: '2.5',
    4: '3.0',
    5: '3.5',
    6: '4.0',
    7: '4.5',
    8: '5.0'
}

# Load your CSV (update path if needed)
input_csv_path = "/kaggle/input/test-transcription/transcribed_test.csv"
df = pd.read_csv(input_csv_path)

# Make sure the relevant columns exist
assert 'filename' in df.columns and 'transcription' in df.columns, "Missing required columns."

# Store predictions
results = []

# Inference loop


In [6]:
# Inference loop
for idx, row in tqdm(df.iterrows(), total=len(df)):
    text = row['transcription']
    filename = row['filename']

    # Fallback for invalid transcription
    if not isinstance(text, str) or text.strip() == "":
        results.append({
            "filename": filename,
            "predicted_label": 5  # Fallback label for invalid input
        })
        continue

    try:
        # Tokenize and predict
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            logits = model(**inputs).logits
            probs = F.softmax(logits, dim=1)
            pred_class = torch.argmax(probs, dim=1).item()
            label = id_to_label[pred_class]

        results.append({
            "filename": filename,
            "predicted_label": label
        })

    except Exception as e:
        print(f"Error on row {idx}: {e}. Assigning fallback label 5.")
        results.append({
            "filename": filename,
            "predicted_label": 5  # Also use fallback in case of unexpected error
        })

# Save predictions
output_df = pd.DataFrame(results)
output_df.to_csv("/kaggle/working/predicted_labels1.csv", index=False)

print("✅ Predictions saved to /kaggle/working/predicted_labels.csv")


100%|██████████| 195/195 [00:12<00:00, 15.63it/s]

✅ Predictions saved to /kaggle/working/predicted_labels.csv
